In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adobe-fivek/b/a1380-jmac_DSC0493.jpg
/kaggle/input/adobe-fivek/b/a3202-07-06-02-at-13h18m43-s_MG_1425.jpg
/kaggle/input/adobe-fivek/b/a3683-KE_-0037.jpg
/kaggle/input/adobe-fivek/b/a0619-20081019_at_01h22m56__MG_3327.jpg
/kaggle/input/adobe-fivek/b/a4881-Duggan_090405_7225.jpg
/kaggle/input/adobe-fivek/b/a2694-IMG_2050.jpg
/kaggle/input/adobe-fivek/b/a1852-_DSC8964.jpg
/kaggle/input/adobe-fivek/b/a4603-Duggan_050714_1956.jpg
/kaggle/input/adobe-fivek/b/a4801-_DGW0327.jpg
/kaggle/input/adobe-fivek/b/a3438-kme_193.jpg
/kaggle/input/adobe-fivek/b/a2612-dgw_115.jpg
/kaggle/input/adobe-fivek/b/a4165-jmacdscf0032.jpg
/kaggle/input/adobe-fivek/b/a0842-tc_DSC2360.jpg
/kaggle/input/adobe-fivek/b/a2957-KE_-2762.jpg
/kaggle/input/adobe-fivek/b/a2874-IMG_4365.jpg
/kaggle/input/adobe-fivek/b/a1601-IMG_4767.jpg
/kaggle/input/adobe-fivek/b/a0457-NKIM_099.jpg
/kaggle/input/adobe-fivek/b/a3851-IMG_3430.jpg
/kaggle/input/adobe-fivek/b/a1552-jmac_DSC3309.jpg
/kaggle/input/adobe-fivek/b/a337

In [2]:
# pip install pytorch-msssim

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import numpy as np
from PIL import Image
import math

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetBackbone(nn.Module):
    def __init__(self, in_channels=3, base_channels=64):
        super().__init__()
        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels, base_channels, 3, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(base_channels, base_channels, 3, padding=1),
            nn.LeakyReLU()
        )
        self.pool = nn.MaxPool2d(2)
        
        self.enc2 = nn.Sequential(
            nn.Conv2d(base_channels, base_channels*2, 3, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(base_channels*2, base_channels*2, 3, padding=1),
            nn.LeakyReLU()
        )
        
        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(base_channels*2, base_channels*4, 3, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(base_channels*4, base_channels*4, 3, padding=1),
            nn.LeakyReLU()
        )
        
        # Decoder
        self.up2 = nn.ConvTranspose2d(base_channels*4, base_channels*2, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(base_channels*4, base_channels*2, 3, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(base_channels*2, base_channels*2, 3, padding=1),
            nn.LeakyReLU()
        )
        
        self.up1 = nn.ConvTranspose2d(base_channels*2, base_channels, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(base_channels*2, base_channels, 3, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(base_channels, base_channels, 3, padding=1),
            nn.LeakyReLU()
        )
        
        # Output feature map
        self.out_conv = nn.Conv2d(base_channels, base_channels, 1)
        
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        
        # Bottleneck
        b = self.bottleneck(self.pool(e2))
        
        # Decoder
        d2 = self.dec2(torch.cat([self.up2(b), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))
        
        out = self.out_conv(d1)  # Feature map
        # Concatenate input image channels if needed
        out = torch.cat([x, out], dim=1)  # C = 3 + C'
        return out


In [5]:
# class FilterParamPredictor(nn.Module):
def __init__(self, in_channels, out_params, dropout=0.5):

        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels, in_channels*2, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels*2, in_channels*4, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2)
        )

        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)

        # Fully Connected Layer
        self.fc = nn.Sequential(
            nn.Linear(in_channels*4, out_params),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        def forward(self, x):
            x = self.conv_block(x)
            x = self.gap(x)            # [B, C, 1, 1]
            x = x.view(x.size(0), -1)  # Flatten [B, C]
            out = self.fc(x)           # [B, out_params]
            return out


In [6]:
class CubicPolynomialFilter(nn.Module):
    def __init__(self, num_channels=3):
        super().__init__()
        self.num_channels = num_channels
        self.num_params = 10  # cubic-10
        
        # Initialize parameters with better values
        self.params = nn.Parameter(torch.zeros(num_channels, self.num_params))
        with torch.no_grad():
            # Initialize bias term (J) to 1.0 for identity-like behavior
            self.params[:, -1] = 1.0  # J parameter
            # Small random values for other parameters
            self.params[:, :-1] = torch.randn(num_channels, self.num_params-1) * 0.1
        
    def forward(self, x):
        """
        x: input image tensor [B, C, H, W]
        """
        B, C, H, W = x.shape
        device = x.device
        
        # Create normalized coordinate grids [-1,1]
        y_coords, x_coords = torch.meshgrid(
            torch.linspace(-1, 1, H, device=device),
            torch.linspace(-1, 1, W, device=device),
            indexing='ij'
        )
        y_coords = y_coords.float().unsqueeze(0).unsqueeze(0)  # [1,1,H,W]
        x_coords = x_coords.float().unsqueeze(0).unsqueeze(0)  # [1,1,H,W]
        
        out = torch.zeros_like(x)
        
        for ch in range(C):
            p = self.params[ch]  # [10]
            i = x[:, ch:ch+1, :, :]  # [B,1,H,W]
            
            # cubic-10 formula: i * (Ax³ + Bx²y + Cx² + D + Exy + Fx + Gy³ + Hy² + Iy + J)
            A, B, C_, D, E, F, G, H_, I_, J = p
            poly = i * (A*x_coords**3 + B*x_coords**2*y_coords + C_*x_coords**2 + D + 
                       E*x_coords*y_coords + F*x_coords + G*y_coords**3 + H_*y_coords**2 + 
                       I_*y_coords + J)
            
            out[:, ch:ch+1, :, :] = poly
        
        return out


In [7]:
class EllipticalFilter(nn.Module):
    def __init__(self, num_channels=None):
        super().__init__()
        self.num_channels = num_channels  
        self.params = None  

    def forward(self, x):
        B, C, H, W = x.shape
        device = x.device

        if self.params is None or self.num_channels != C:
            self.num_channels = C
            self.params = nn.Parameter(torch.zeros(C, 6, device=device))
            with torch.no_grad():
                self.params[:, 0] = H // 2  # h (center_x)
                self.params[:, 1] = W // 2  # k (center_y) 
                self.params[:, 2] = H // 4  # a (semi-axis)
                self.params[:, 3] = W // 4  # b (semi-axis)
                self.params[:, 4] = 0.0     # theta (rotation)
                self.params[:, 5] = 1.0     # se (scale)

        # Clamp parameters to reasonable ranges to prevent NaN
        with torch.no_grad():
            self.params.data = torch.clamp(self.params.data, -1000, 1000)

        y_coords, x_coords = torch.meshgrid(
            torch.arange(H, device=device),
            torch.arange(W, device=device),
            indexing='ij'  
        )
        y_coords = y_coords.float().unsqueeze(0).unsqueeze(0)
        x_coords = x_coords.float().unsqueeze(0).unsqueeze(0)

        out = torch.zeros_like(x)

        for ch in range(C):
            h, k, a, b, theta, se = self.params[ch]
            
            # Clamp parameters to prevent division by zero and extreme values
            a = torch.clamp(torch.abs(a), min=1.0)
            b = torch.clamp(torch.abs(b), min=1.0) 
            se = torch.clamp(se, 0.0, 2.0)
            
            cos_theta = torch.cos(theta)
            sin_theta = torch.sin(theta)
            x_rot = (x_coords - h) * cos_theta + (y_coords - k) * sin_theta
            y_rot = (x_coords - h) * sin_theta - (y_coords - k) * cos_theta
            dist = (x_rot**2 / (a**2)) + (y_rot**2 / (b**2))
            scale = se * torch.clamp(1 - dist, min=0.0, max=1.0)
            
            # Add small bias to prevent complete zero outputs
            out[:, ch:ch+1, :, :] = x[:, ch:ch+1, :, :] * (scale + 0.1)

        return torch.clamp(out, 0.0, 1.0)


class GraduatedFilter(nn.Module):
    def __init__(self, num_channels=None):
        super().__init__()
        self.num_channels = num_channels
        self.params = None

    def forward(self, x):
        B, C, H, W = x.shape
        device = x.device

        if self.params is None or self.num_channels != C:
            self.num_channels = C
            self.params = nn.Parameter(torch.zeros(C, 6, device=device))
            with torch.no_grad():
                self.params[:, 0] = 1.0     # m (slope)
                self.params[:, 1] = H // 2  # c (intercept)  
                self.params[:, 2] = H // 4  # o1 (distance1)
                self.params[:, 3] = 3*H//4  # o2 (distance2)
                self.params[:, 4] = 1.0     # sg (scale)
                self.params[:, 5] = 0.0     # ginv (invert)

        # Clamp parameters to prevent NaN
        with torch.no_grad():
            self.params.data = torch.clamp(self.params.data, -1000, 1000)

        y_coords, x_coords = torch.meshgrid(
            torch.arange(H, device=device),
            torch.arange(W, device=device),
            indexing='ij'
        )
        y_coords = y_coords.float().unsqueeze(0).unsqueeze(0)
        x_coords = x_coords.float().unsqueeze(0).unsqueeze(0)

        out = torch.zeros_like(x)

        for ch in range(C):
            m, c, o1, o2, sg, ginv_real = self.params[ch]
            
            # Clamp parameters to safe ranges
            m = torch.clamp(m, -10, 10)
            o1 = torch.clamp(torch.abs(o1), min=1.0)
            o2 = torch.clamp(torch.abs(o2), min=1.0)
            sg = torch.clamp(sg, 0.0, 2.0)
            
            ginv = torch.sigmoid(ginv_real)  # Use sigmoid instead of sign for stability
            alpha = torch.atan(m)
            l = y_coords - (m * x_coords + c)
            d1 = o1 * torch.abs(torch.cos(alpha))
            d2 = o2 * torch.abs(torch.cos(alpha))
            
            a = sg * torch.clamp(0.5 * (1 + l/d2), 0.0, 1.0)
            b = sg * torch.clamp(0.5 * (1 + l/d1), 0.0, 1.0)
            s = torch.where(l >= 0, ginv*a + (1-ginv)*b, (1-ginv)*a + ginv*b)
            
            # Add small bias to prevent complete zero outputs
            s = torch.clamp(s + 0.1, 0.0, 1.0)
            
            out[:, ch:ch+1, :, :] = x[:, ch:ch+1, :, :] * s

        return torch.clamp(out, 0.0, 1.0)

In [8]:
class DeepLPF(nn.Module):
    def __init__(self, in_channels=3, base_channels=64):
        super().__init__()
        self.backbone = UNetBackbone(in_channels=in_channels, base_channels=base_channels)
        self.polynomial_filter = CubicPolynomialFilter(num_channels=in_channels)
        self.graduated_filter = GraduatedFilter()  
        self.elliptical_filter = EllipticalFilter() 

    def forward(self, x):
        feature_map = self.backbone(x)
        Y1 = feature_map[:, :3, :, :]  # Backbone-enhanced image
        Y2 = self.polynomial_filter(Y1)

        C_prime_features = feature_map[:, 3:, :, :]
        two_stream_input = torch.cat([Y2, C_prime_features], dim=1) if C_prime_features.shape[1] > 0 else Y2

        S_elliptical = self.elliptical_filter(two_stream_input)
        S_graduated  = self.graduated_filter(two_stream_input)
        S = S_elliptical + S_graduated

        # Use only first 3 channels of scaling map to match Y2
        Y3 = Y2 * S[:, :3, :, :]

        Y_hat = Y3 + Y1
        return Y_hat


In [9]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Dataset
# -----------------------------
class FiveKDataset(Dataset):
    def __init__(self, root_input, root_target, transform=None):
        self.input_files = sorted([os.path.join(root_input, f) for f in os.listdir(root_input)])
        self.target_files = sorted([os.path.join(root_target, f) for f in os.listdir(root_target)])
        self.transform = transform

    def __len__(self):
        return len(self.input_files)

    def __getitem__(self, idx):
        x = Image.open(self.input_files[idx]).convert("RGB")
        y = Image.open(self.target_files[idx]).convert("RGB")

        if self.transform:
            x = self.transform(x)
            y = self.transform(y)
        return x, y

transform = transforms.Compose([
    transforms.ToTensor() , # [0,1]
    transforms.Resize((512, 512)), 
])

dataset = FiveKDataset(root_input="/kaggle/input/adobe-fivek/c", root_target="/kaggle/input/adobe-fivek/raw/", transform=transform)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [10]:
import torch
import torch.nn as nn

# Simplified and stable loss function
class SimpleLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.L1Loss()
        self.mse = nn.MSELoss()
    
    def forward(self, y_hat, y):
        # Clamp outputs to valid range to prevent NaN
        y_hat = torch.clamp(y_hat, 0.0, 1.0)
        y = torch.clamp(y, 0.0, 1.0)
        
        l1_loss = self.l1(y_hat, y)
        mse_loss = self.mse(y_hat, y)
        
        return l1_loss + 0.1 * mse_loss

# Alternative: Stable DeepLPFLoss if you want to use Lab color space
class StableDeepLPFLoss(nn.Module):
    def __init__(self, w_lab=1.0, w_ssim=0.5):
        super().__init__()
        self.w_lab = w_lab
        self.w_ssim = w_ssim
        self.l1 = nn.L1Loss()
        self.mse = nn.MSELoss()

    def rgb_to_lab(self, img):
        """
        Simplified RGB to Lab conversion to avoid NaN
        img: [B,3,H,W], float32, [0,1]
        """
        # Clamp input to valid range
        img = torch.clamp(img, 0.0, 1.0)
        
        # Simple approximation to avoid complex conversions
        r, g, b = img[:, 0:1], img[:, 1:2], img[:, 2:3]
        
        # Luminance approximation
        L = 0.299 * r + 0.587 * g + 0.114 * b
        
        # Color channels (normalized to [0,1])
        a = 0.5 + 0.5 * (r - g)  # Red-Green
        b_ch = 0.5 + 0.5 * (g - b)  # Yellow-Blue
        
        lab = torch.cat([L, a, b_ch], dim=1)
        return torch.clamp(lab, 0.0, 1.0)

    def ssim_loss(self, y_hat, y, window_size=11):
        """Simple SSIM implementation"""
        def gaussian_kernel(size, sigma=1.5):
            coords = torch.arange(size, dtype=torch.float32) - size // 2
            g = torch.exp(-(coords ** 2) / (2 * sigma ** 2))
            g = g / g.sum()
            return g.view(1, 1, size, 1) * g.view(1, 1, 1, size)
        
        kernel = gaussian_kernel(window_size).to(y.device)
        mu1 = torch.nn.functional.conv2d(y_hat, kernel, padding=window_size//2)
        mu2 = torch.nn.functional.conv2d(y, kernel, padding=window_size//2)
        
        mu1_sq = mu1 ** 2
        mu2_sq = mu2 ** 2
        mu1_mu2 = mu1 * mu2
        
        sigma1_sq = torch.nn.functional.conv2d(y_hat * y_hat, kernel, padding=window_size//2) - mu1_sq
        sigma2_sq = torch.nn.functional.conv2d(y * y, kernel, padding=window_size//2) - mu2_sq
        sigma12 = torch.nn.functional.conv2d(y_hat * y, kernel, padding=window_size//2) - mu1_mu2
        
        C1 = 0.01 ** 2
        C2 = 0.03 ** 2
        
        ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
        return ssim_map.mean()

    def forward(self, y_hat, y):
        """
        y_hat, y: [B,3,H,W], float32, [0,1]
        """
        # Clamp inputs
        y_hat = torch.clamp(y_hat, 0.0, 1.0)
        y = torch.clamp(y, 0.0, 1.0)
        
        # Convert to Lab
        y_lab = self.rgb_to_lab(y)
        y_hat_lab = self.rgb_to_lab(y_hat)

        # Chrominance loss (L1 on a,b channels)
        chroma_loss = self.l1(y_hat_lab[:,1:3,:,:], y_lab[:,1:3,:,:])

        # Luminance loss (1 - SSIM on L channel)
        ssim_val = self.ssim_loss(y_hat_lab[:,0:1,:,:], y_lab[:,0:1,:,:])
        luminance_loss = 1 - torch.clamp(ssim_val, 0.0, 1.0)

        total_loss = self.w_lab * chroma_loss + self.w_ssim * luminance_loss
        
        # Check for NaN and return fallback loss if needed
        if torch.isnan(total_loss):
            return self.mse(y_hat, y)
        
        return total_loss

In [11]:
from torch.utils.data import random_split, DataLoader
dataset = FiveKDataset(
    root_input="/kaggle/input/adobe-fivek/c",   
    root_target="/kaggle/input/adobe-fivek/raw", 
    transform=transform
)
dataset_size = len(dataset)  # Adobe FiveK Dataset
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [12]:
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.path)
        if self.verbose:
            print(f"Validation loss decreased. Saving model to {self.path}")


In [ ]:
import torch
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model
model = DeepLPF()  
model.to(device)

# Loss - Use SimpleLoss instead of complex DeepLPFLoss to avoid NaN
criterion = SimpleLoss()  # Changed to stable loss function
# Alternative: criterion = StableDeepLPFLoss(w_lab=1.0, w_ssim=0.5)

# Optimizer with lower learning rate for stability
optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)

# Early stopping
early_stopping = EarlyStopping(patience=3, verbose=True, path='deep_lpf_best.pt')

num_epochs = 5
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # --------------------
    # Training
    # --------------------
    model.train()
    train_loss = 0.0
    num_batches = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, targets = batch  
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        # Clamp inputs to valid range
        inputs = torch.clamp(inputs, 0.0, 1.0)
        targets = torch.clamp(targets, 0.0, 1.0)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Clamp outputs to prevent extreme values
        outputs = torch.clamp(outputs, 0.0, 1.0)
        
        loss = criterion(outputs, targets)
        
        # Check for NaN loss and skip if found
        if torch.isnan(loss):
            print(f"NaN loss detected, skipping batch")
            continue
            
        loss.backward()
        
        # Add gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()

        train_loss += loss.item()
        num_batches += 1
    
    if num_batches > 0:
        train_loss /= num_batches
    else:
        train_loss = float('inf')

    # --------------------
    # Validation
    # --------------------
    model.eval()
    val_loss = 0.0
    num_val_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Clamp inputs
            inputs = torch.clamp(inputs, 0.0, 1.0)
            targets = torch.clamp(targets, 0.0, 1.0)

            outputs = model(inputs)
            outputs = torch.clamp(outputs, 0.0, 1.0)
            
            loss = criterion(outputs, targets)
            
            if not torch.isnan(loss):
                val_loss += loss.item()
                num_val_batches += 1

    if num_val_batches > 0:
        val_loss /= num_val_batches
    else:
        val_loss = float('inf')

    # Store losses for plotting
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f'Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}')

    # Early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

# Load best model
model.load_state_dict(torch.load('deep_lpf_best.pt'))

# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training History')
plt.show()

Epoch 1/5 - Training:   7%|▋         | 62/875 [04:41<1:01:34,  4.54s/it]

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def test_model_predictions(model, test_loader, device=None, ckpt_path=None, num_samples=4):

    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')

    # Load checkpoint if provided (handles both plain state_dict and checkpoint dicts)
    if ckpt_path is not None:
        ckpt = torch.load(ckpt_path, map_location=device)
        if isinstance(ckpt, dict):
            if 'model' in ckpt:
                state_dict = ckpt['model']
            elif 'state_dict' in ckpt:
                state_dict = ckpt['state_dict']
            else:
                state_dict = ckpt  # assume it's a state_dict-like dict
        else:
            state_dict = ckpt
        model.load_state_dict(state_dict, strict=False)

    model = model.to(device)
    model.eval()

    # Iterate over test loader and visualize only the first batch
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            # use only first batch
            inputs = inputs.to(device)
            targets = targets.to(device)

            inputs = torch.clamp(inputs, 0.0, 1.0)
            targets = torch.clamp(targets, 0.0, 1.0)

            outputs = model(inputs)
            outputs = torch.clamp(outputs, 0.0, 1.0)

            # Move to CPU for visualization/statistics
            inputs_cpu = inputs.detach().cpu()
            targets_cpu = targets.detach().cpu()
            outputs_cpu = outputs.detach().cpu()

            bs = inputs_cpu.size(0)
            n = min(num_samples, bs)

            fig, axes = plt.subplots(3, n, figsize=(4 * n, 12))
            # ensure axes is indexable as axes[row, col]
            if n == 1:
                # axes will be shape (3,) in this case -> convert to (3,1)
                axes = np.expand_dims(axes, axis=1)

            for i in range(n):
                in_img = np.clip(inputs_cpu[i].permute(1, 2, 0).numpy(), 0.0, 1.0)
                tar_img = np.clip(targets_cpu[i].permute(1, 2, 0).numpy(), 0.0, 1.0)
                out_img = np.clip(outputs_cpu[i].permute(1, 2, 0).numpy(), 0.0, 1.0)

                axes[0, i].imshow(in_img)
                axes[0, i].set_title(f'Input {i+1}')
                axes[0, i].axis('off')

                axes[1, i].imshow(tar_img)
                axes[1, i].set_title(f'Target {i+1}')
                axes[1, i].axis('off')

                axes[2, i].imshow(out_img)
                axes[2, i].set_title(f'Enhanced {i+1}')
                axes[2, i].axis('off')

            plt.tight_layout()
            plt.show()

            # statistics (use CPU tensors)
            out_min = float(outputs_cpu.min().item())
            out_max = float(outputs_cpu.max().item())
            out_mean = float(outputs_cpu.mean().item())
            out_std = float(outputs_cpu.std().item())

            print("\n=== Model Output Statistics (first batch) ===")
            print(f"Output range: [{out_min:.3f}, {out_max:.3f}]")
            print(f"Output mean: {out_mean:.3f}")
            print(f"Output std: {out_std:.3f}")

            if out_mean < 0.05:
                print("⚠️ WARNING: Outputs are mostly very dark (mean < 0.05).")
            else:
                print("✅ Outputs appear reasonable (mean >= 0.05).")

            break  # only visualize first batch

print("Testing the trained model...")
test_model_predictions(model, test_loader, device, num_samples=10)

In [ ]:
# Test the trained model and visualize results
import matplotlib.pyplot as plt
import numpy as np

def test_model_predictions(model, test_loader, device, num_samples=4):
    """
    Test the model and display input/output comparisons
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = DeepLPF()
    model.to(device)
    model.load_state_dict(torch.load('/kaggle/input/deeplpf/pytorch/default/1/deep_lpf_best.pt') , strict=False)
    model.eval()
    
    # Get a batch of test data
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            if batch_idx == 0:  # Use first batch
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                # Clamp inputs
                inputs = torch.clamp(inputs, 0.0, 1.0)
                targets = torch.clamp(targets, 0.0, 1.0)
                
                # Get model predictions
                outputs = model(inputs)
                outputs = torch.clamp(outputs, 0.0, 1.0)
                
                # Move to CPU for visualization
                inputs_cpu = inputs.cpu()
                targets_cpu = targets.cpu()
                outputs_cpu = outputs.cpu()
                
                # Create visualization
                fig, axes = plt.subplots(3, num_samples, figsize=(16, 12))
                
                for i in range(min(num_samples, inputs.size(0))):
                    # Convert from tensor to numpy and transpose for matplotlib
                    input_img = inputs_cpu[i].permute(1, 2, 0).numpy()
                    target_img = targets_cpu[i].permute(1, 2, 0).numpy()
                    output_img = outputs_cpu[i].permute(1, 2, 0).numpy()
                    
                    # Input image
                    axes[0, i].imshow(input_img)
                    axes[0, i].set_title(f'Input {i+1}')
                    axes[0, i].axis('off')
                    
                    # Target image
                    axes[1, i].imshow(target_img)
                    axes[1, i].set_title(f'Target {i+1}')
                    axes[1, i].axis('off')
                    
                    # Model output
                    axes[2, i].imshow(output_img)
                    axes[2, i].set_title(f'Enhanced {i+1}')
                    axes[2, i].axis('off')
                
                plt.tight_layout()
                plt.show()
                
                # Print some statistics
                print("\n=== Model Output Statistics ===")
                print(f"Output range: [{outputs.min().item():.3f}, {outputs.max().item():.3f}]")
                print(f"Output mean: {outputs.mean().item():.3f}")
                print(f"Output std: {outputs.std().item():.3f}")
                
                # Check if outputs are black (all values near 0)
                if outputs.mean().item() < 0.1:
                    print("⚠️  WARNING: Outputs are still mostly black!")
                else:
                    print("✅ Outputs look good - model is producing enhanced images!")
                
                break

# Test the model
print("Testing the trained model...")
test_model_predictions(model, test_loader, device, num_samples=3)

In [ ]:
# Save sample predictions and create before/after comparisons
import torchvision.transforms as transforms
from torchvision.utils import save_image

def save_predictions(model, test_loader, device, save_dir='predictions', num_samples=8):
    """
    Save input, target, and predicted images for visual inspection
    """
    import os
    os.makedirs(save_dir, exist_ok=True)
    
    model.eval()
    saved_count = 0
    
    print(f"Saving predictions to '{save_dir}' directory...")
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            if saved_count >= num_samples:
                break
                
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Clamp inputs
            inputs = torch.clamp(inputs, 0.0, 1.0)
            targets = torch.clamp(targets, 0.0, 1.0)
            
            # Get predictions
            outputs = model(inputs)
            outputs = torch.clamp(outputs, 0.0, 1.0)
            
            # Save each image in the batch
            batch_size = inputs.size(0)
            for i in range(min(batch_size, num_samples - saved_count)):
                # Save individual images
                save_image(inputs[i], f'{save_dir}/input_{saved_count:03d}.png')
                save_image(targets[i], f'{save_dir}/target_{saved_count:03d}.png')
                save_image(outputs[i], f'{save_dir}/enhanced_{saved_count:03d}.png')
                
                # Create comparison image (side by side)
                comparison = torch.cat([inputs[i], targets[i], outputs[i]], dim=2)  # Concatenate along width
                save_image(comparison, f'{save_dir}/comparison_{saved_count:03d}.png')
                
                saved_count += 1
                
                if saved_count >= num_samples:
                    break
    
    print(f"✅ Saved {saved_count} prediction samples")
    print(f"Files saved: input_XXX.png, target_XXX.png, enhanced_XXX.png, comparison_XXX.png")

# Quick single image test
def quick_test():
    """
    Quick test with a single image to check if model works
    """
    print("Running quick test...")
    
    # Create a dummy test image
    test_input = torch.randn(1, 3, 256, 256).to(device)
    test_input = torch.clamp(test_input * 0.5 + 0.5, 0.0, 1.0)  # Normalize to [0,1]
    
    model.eval()
    with torch.no_grad():
        test_output = model(test_input)
        test_output = torch.clamp(test_output, 0.0, 1.0)
    
    print(f"✅ Model forward pass successful!")
    print(f"Input shape: {test_input.shape}")
    print(f"Output shape: {test_output.shape}")
    print(f"Output range: [{test_output.min():.3f}, {test_output.max():.3f}]")
    print(f"Output mean: {test_output.mean():.3f}")
    
    return test_input, test_output

# Run quick test
test_input, test_output = quick_test()

# Save real predictions
save_predictions(model, test_loader, device, num_samples=4)